<a href="https://colab.research.google.com/github/tim-fihost/ai-application-/blob/main/week13/week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Tue Nov 14 06:22:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
!pip install --upgrade --quiet pip
!pip install --quiet git+https://github.com/huggingface/transformers.git datasets[audio]


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [12]:
from transformers import MusicgenForConditionalGeneration
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")
{"model_id":"82c102ab57494d0aab4abd55a0f1814b","version_major":2,"version_minor":0}
{"model_id":"ea3122d136ac49f09b4314f967381b08","version_major":2,"version_minor":0}
{"model_id":"4c1933bb2547478bb3af5ddec7673def","version_major":2,"version_minor":0}

{'model_id': '4c1933bb2547478bb3af5ddec7673def',
 'version_major': 2,
 'version_minor': 0}

In [13]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device);

In [14]:
unconditional_inputs = model.get_unconditional_inputs(num_samples=1)
audio_values = model.generate(**unconditional_inputs, do_sample=True, max_new_tokens=256)


In [15]:
from IPython.display import Audio
sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)


In [16]:
import scipy
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate,
data=audio_values[0, 0].cpu().numpy())


In [17]:
audio_length_in_s = 256 / model.config.audio_encoder.frame_rate
audio_length_in_s

5.12

In [20]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
inputs = processor(
 text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
 padding=True,
 return_tensors="pt",
)
audio_values = model.generate(**inputs.to(device), do_sample=True,
guidance_scale=3, max_new_tokens=256)
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)
{"model_id":"1f920f2ef8c844458c556a5776e240c8","version_major":2,"version_minor":0}
{"model_id":"da80246dd846479f8b79ae2956dbed05","version_major":2,"version_minor":0}
{"model_id":"c6a00b70646a46ccad53445ba8b45383","version_major":2,"version_minor":0}
{"model_id":"a76a2ab77f6b404183367034ed4414ec","version_major":2,"version_minor":0}
{"model_id":"f3a2da667da846aa99d34237b39b2f0c","version_major":2,"version_minor":0}


{'model_id': 'f3a2da667da846aa99d34237b39b2f0c',
 'version_major': 2,
 'version_minor': 0}

In [21]:
from datasets import load_dataset
dataset = load_dataset("sanchit-gandhi/gtzan", split="train",
streaming=True)
sample = next(iter(dataset))["audio"]
# take the first half of the audio sample
sample["array"] = sample["array"][: len(sample["array"]) // 2]
inputs = processor(
 audio=sample["array"],
 sampling_rate=sample["sampling_rate"],
 text=["80s blues track with groovy saxophone"],
 padding=True,
 return_tensors="pt",
)
audio_values = model.generate(**inputs.to(device), do_sample=True,
guidance_scale=3, max_new_tokens=256)
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)
{"model_id":"a057a4eb71874800b7bad2959976714b","version_major":2,"version_minor":0}


{'model_id': 'a057a4eb71874800b7bad2959976714b',
 'version_major': 2,
 'version_minor': 0}

In [23]:
sample = next(iter(dataset))["audio"]
# take the first quater of the audio sample
sample_1 = sample["array"][: len(sample["array"]) // 4]
# take the first half of the audio sample
sample_2 = sample["array"][: len(sample["array"]) // 2]
inputs = processor(
 audio=[sample_1, sample_2],
 sampling_rate=sample["sampling_rate"],
 text=["80s blues track with groovy saxophone", "90s rock song with loud guitars and heavy drums"],
 padding=True,
 return_tensors="pt",
)
audio_values = model.generate(**inputs.to(device), do_sample=True,
guidance_scale=3, max_new_tokens=256)
# post-process to remove padding from the batched audio
audio_values = processor.batch_decode(audio_values,
padding_mask=inputs.padding_mask)
Audio(audio_values[0], rate=sampling_rate)

In [24]:
model.generation_config

GenerationConfig {
  "bos_token_id": 2048,
  "decoder_start_token_id": 2048,
  "do_sample": true,
  "guidance_scale": 3.0,
  "max_length": 1500,
  "pad_token_id": 2048
}

In [25]:
# increase the guidance scale to 4.0
model.generation_config.guidance_scale = 4.0
# set the max new tokens to 256
model.generation_config.max_new_tokens = 256
# set the softmax sampling temperature to 1.5
model.generation_config.temperature = 1.5

In [26]:
audio_values = model.generate(**inputs.to(device))